# **Homework**: dlt workshop

Homework submission link: https://courses.datatalks.club/de-zoomcamp-2024/homework/workshop1

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

In [2]:
def square_root_generator(limit):
  n = 1
  while n <= limit:
    yield n ** 0.5
    n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
  print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


### Question 1:

What is the sum of the outputs of the generator for limit = 5?

- 10.23433234744176
- 7.892332347441762
- **8.382332347441762**    <-- answer
- 9.123332347441762

In [3]:
sum_outputs = 0

generator = square_root_generator(limit=5)

for sqrt_value in generator:
  sum_outputs+=sqrt_value

print(sum_outputs)

8.382332347441762


### Question 2:

What is the 13th number yielded?

- 4.236551275463989
- **3.605551275463989**    <-- answer
- 2.345551275463989
- 5.678551275463989

In [4]:
generator = square_root_generator(limit=13)

for sqrt_value in generator:
  print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data

Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. After correctly appending the data, calculate the sum of all ages of people.

In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


### Question 3:

Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

- **353**    <-- answer
- 365
- 378
- 390

In [6]:
import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(), table_name="people", write_disposition="replace")
print(info)

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

print("\nSum of ages:")
sum_ages = conn.sql("SELECT SUM(Age) FROM generators.people").df()
display(sum_ages)

Pipeline dlt_colab_kernel_launcher load step completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707232767.5042026 is LOADED and contains no failed jobs

Sum of ages:


,sum(Age)
0,140.0


In [7]:
info = generators_pipeline.run(people_2(), table_name="people", write_disposition="append")
print(info)

print("\nSum of ages:")
sum_ages = conn.sql("SELECT SUM(Age) FROM generators.people").df()
display(sum_ages)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707232768.3453343 is LOADED and contains no failed jobs

Sum of ages:


,sum(Age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

### Question 4

Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

- 215
- **266**    <-- answer
- 241
- 258

In [8]:
info = generators_pipeline.run(people_1(), table_name="people_merge", write_disposition="replace", primary_key='ID')
print(info)

info = generators_pipeline.run(people_2(), table_name="people_merge", write_disposition="merge", primary_key='ID')
print(info)

print("\nTable:")
table = conn.sql("SELECT * FROM generators.people_merge").df()
display(table)

print("\nSum of ages:")
sum_ages = conn.sql("SELECT SUM(Age) FROM generators.people_merge").df()
display(sum_ages)

Pipeline dlt_colab_kernel_launcher load step completed in 0.29 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707232778.9422414 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707232779.7298532 is LOADED and contains no failed jobs

Table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707232778.9422414,ibMJ8CJ603KtlA,None
1,2,Person_2,27,City_A,1707232778.9422414,GXdiHyoVM4fD5w,None
2,5,Person_5,35,City_B,1707232779.7298532,HSsUnbGEfLBJug,Job_5
3,7,Person_7,37,City_B,1707232779.7298532,fsTajL6doj6nKQ,Job_7
4,4,Person_4,34,City_B,1707232779.7298532,UVt8JoemD8u6Lw,Job_4
5,3,Person_3,33,City_B,1707232779.7298532,xhsFpR0Z+7OwUw,Job_3
6,6,Person_6,36,City_B,1707232779.7298532,56q+HDO0ruYLDQ,Job_6
7,8,Person_8,38,City_B,1707232779.7298532,TOQleR0TK4CAxg,Job_8



Sum of ages:


,sum(Age)
0,266.0
